Импортируем библиотеки

In [1]:
import requests as req
import bs4  as bs
from collections import Counter as c

Определяем константы и перемнные

In [2]:
URL_SEARCH = "https://hh.ru/search/vacancy?text=python+middle"
URL_HEADERS = {'User-Agent': 'Mozilla/5.0'}

In [3]:
numersOfPage = 0
listLinkOfVacancy = []
listOfRequirements = []
url_params = {'page':0}

Определяем функции

In [4]:
def getBSPage(URL_SEARCH):
    result = req.get(URL_SEARCH, headers=URL_HEADERS, params=url_params)
    return bs.BeautifulSoup(result.content,'lxml')

In [5]:
def addVacancyToList(soup):
    for vac in list(soup.find_all('a',{'data-qa': 'serp-item__title','target':'_blank'})):
        vac = str(vac)
        if vac.find('hh.ru/vacancy/') > 0 :
            listLinkOfVacancy.append('https://' + vac[vac.find('hh.ru/vacancy/'):vac.find('?')])

Собираем информацию о всех вакансиях

In [6]:
# Получаем кол-во страниц с результатами поиска
soup = getBSPage(URL_SEARCH)
subst = str(list(soup.find_all('a',{'data-qa': 'pager-page'}))[-1]).split(';')[1]
numers_of_page = int(subst[subst.find('=')+1:subst.find('&')])
for i in range(numers_of_page):
    if i == 0:
        addVacancyToList(soup)
    else:
        url_params['page'] = i
        soup = getBSPage(URL_SEARCH)
        addVacancyToList(soup)

Собираем информацию о требованиях по вакансиям

In [7]:
for link in listLinkOfVacancy:
    soup = getBSPage(link)
    vacancies = soup.find_all('div', {'class': 'vacancy-section'})
    for vacancy in vacancies:
        if vacancy.find('h2',{'class':'bloko-header-section-2'}):
            requirements = vacancy.find_all('span')
            for requirement in requirements:
                listOfRequirements.append(requirement.text)

Выводим результат

In [8]:
print (f'Популярные требования {c(listOfRequirements).most_common(10)}')

Популярные требования [('Python', 197), ('SQL', 110), ('Git', 89), ('Linux', 82), ('PostgreSQL', 81), ('Docker', 64), ('Django Framework', 36), ('Kubernetes', 30), ('RabbitMQ', 24), ('Redis', 22)]
